In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

import os


In [3]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test  = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub   = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')

train.shape, test.shape, sub.shape


((28984, 8), (4142, 5), (4142, 2))

In [4]:
train['sex'] = train['sex'].fillna('na')
train['age_approx'] = train['age_approx'].fillna(0)
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('na')

test['sex'] = test['sex'].fillna('na')
test['age_approx'] = test['age_approx'].fillna(0)
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('na')


In [5]:
train.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
3,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
4,ISIC_0074311,IP_2950485,female,40.0,lower extremity,unknown,benign,0


In [6]:
test.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052212,IP_2842074,female,50.0,lower extremity
1,ISIC_0076545,IP_9802602,male,55.0,upper extremity
2,ISIC_0085172,IP_1705144,female,50.0,lower extremity
3,ISIC_0086709,IP_4109313,male,30.0,torso
4,ISIC_0109568,IP_0825081,male,80.0,lower extremity


In [7]:
train['ID'] = train['image_name'].apply(lambda x: int(x.replace('ISIC_','')))
train['ID_rank'] = train.sort_values('ID').groupby('patient_id')['ID'].rank()

test['ID'] = test['image_name'].apply(lambda x: int(x.replace('ISIC_','')))
test['ID_rank'] = test.sort_values('ID').groupby('patient_id')['ID'].rank()


In [8]:
L = 15
feat = ['sex','age_approx','anatom_site_general_challenge', 'ID_rank']

M = train.target.mean()
te = train.groupby(feat)['target'].agg(['mean','count']).reset_index()
te['ll'] = ((te['mean']*te['count'])+(M*L))/(te['count']+L)
del te['mean'], te['count']

test = test.merge( te, on=feat, how='left' )
test['ll'] = test['ll'].fillna(M)

test.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,ID,ID_rank,ll
0,ISIC_0052212,IP_2842074,female,50.0,lower extremity,52212,1.0,0.004577
1,ISIC_0076545,IP_9802602,male,55.0,upper extremity,76545,1.0,0.009833
2,ISIC_0085172,IP_1705144,female,50.0,lower extremity,85172,1.0,0.004577
3,ISIC_0086709,IP_4109313,male,30.0,torso,86709,1.0,0.056637
4,ISIC_0109568,IP_0825081,male,80.0,lower extremity,109568,1.0,0.011062


In [9]:
sub.target = test.ll.values
sub.head(10)


,image_name,target
0,ISIC_0052212,0.004577
1,ISIC_0076545,0.009833
2,ISIC_0085172,0.004577
3,ISIC_0086709,0.056637
4,ISIC_0109568,0.011062
5,ISIC_0150363,0.003123
6,ISIC_0152823,0.004741
7,ISIC_0154253,0.056637
8,ISIC_0154872,0.011062
9,ISIC_0155273,0.006034


In [10]:
sub.to_csv( 'submission.csv', index=False )
